#Notación Polaca
La notación Polaca o notación de postfijo es una forma inequívoca de escribir una expresión aritmética sin paréntesis. Se define de manera que si

```
exp1 op exp2
```

es una expresión normal completamente entre paréntesis cuya operación es op, entonces la versión de postfijo de esto es


```
pexp1 pexp2 op
```

donde pexp1 es la versión postfijo de exp1 y pexp2 es la versión postfijo de exp2. La versión de postfijo de un solo número es solo ese número. Entonces, por ejemplo, la versión postfix de

((5+2)*(8-3))^4

es

5 2 + 8 3 - * 4 ^

Escriba un programa, **usando la implementación de [Array](https://colab.research.google.com/drive/1yX8PCBgDwM56eaa5q1-YLpge2mBfsu65?usp=sharing)**, que evalúe una expresión que se ingresa en notación de postfijo. Su programa debe admitir la entrada de números como enteros o (coma flotante) dobles, y debe admitir los siguientes operadores:
* "+" para la suma
* "-" para la resta
* "*" para la multiplicación
* "/" para la división
* "^" para exponenciación


In [ ]:
# Copyright 2013, Michael H. Goldwasser
#
# Developed for use with the book:
#
#    Data Structures and Algorithms in Python
#    Michael T. Goodrich, Roberto Tamassia, and Michael H. Goldwasser
#    John Wiley & Sons, 2013
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.

class Empty(Exception):
    pass

import ctypes                                      # provides low-level arrays

class DynamicArray:
  """A dynamic array class akin to a simplified Python list."""

  def __init__(self):
    """Create an empty array."""
    self._n = 0                                    # count actual elements
    self._capacity = 1                             # default array capacity
    self._A = self._make_array(self._capacity)     # low-level array

  def __len__(self):
    """Return number of elements stored in the array."""
    return self._n

  def __getitem__(self, k):
    """Return element at index k."""
    if not 0 <= k < self._n:
      raise IndexError('invalid index')
    return self._A[k]                              # retrieve from array

  def append(self, obj):
    """Add object to end of the array."""
    if self._n == self._capacity:                  # not enough room
      self._resize(2 * self._capacity)             # so double capacity
    self._A[self._n] = obj
    self._n += 1

  def _resize(self, c):                            # nonpublic utitity
    """Resize internal array to capacity c."""
    B = self._make_array(c)                        # new (bigger) array
    for k in range(self._n):                       # for each existing value
      B[k] = self._A[k]
    self._A = B                                    # use the bigger array
    self._capacity = c

  def _make_array(self, c):                        # nonpublic utitity
     """Return new array with capacity c."""
     return (c * ctypes.py_object)()               # see ctypes documentation

  def insert(self, k, value):
    """Insert value at index k, shifting subsequent values rightward."""
    # (for simplicity, we assume 0 <= k <= n in this verion)
    if self._n == self._capacity:                  # not enough room
      self._resize(2 * self._capacity)             # so double capacity
    for j in range(self._n, k, -1):                # shift rightmost first
      self._A[j] = self._A[j-1]
    self._A[k] = value                             # store newest element
    self._n += 1

  def remove(self, value):
    """Remove first occurrence of value (or raise ValueError)."""
    # note: we do not consider shrinking the dynamic array in this version
    for k in range(self._n):
      if self._A[k] == value:              # found a match!
        for j in range(k, self._n - 1):    # shift others to fill gap
          self._A[j] = self._A[j+1]
        self._A[self._n - 1] = None        # help garbage collection
        self._n -= 1                       # we have one less item
        return                             # exit immediately
    raise ValueError('value not found')    # only reached if no match

  def is_empty(self):
    return self._n == 0

  def pop(self):
    """
    Quita el último elemento del Dynamic Array y lo retorna
    La excepción que usa es específica para este proyecto. Si no hay
    elementos en el Dynamic Array, la operación no se puede hacer
    """
    if self.is_empty():
      raise Empty('El Dynamic Array está vacío. La operación en Notación Polaca no se puede realizar')

    last_value = self.__getitem__(self.__len__() - 1)
    self._A[self._n - 1] = None
    self._n -= 1
    return last_value

  def __repr__(self):
    """
    Muestra los elementos en el Dynamic Array (ignora los valores None)
    """
    return repr(self._A[:self.__len__()])

In [ ]:
#Funciones con las operaciones incluidas en la Notación polaca
def division(x, y):
  if y == 0:
    raise Exception('No es válida la división entre cero')
  return x/y

def potencia(x, y):
  if x < 0 and (y == 1/2 or y == 1/4):
    raise Exception('No se le puede sacar raíz par un número negativo')
  return x**y

#Diccionario de las funciones para operar dos valores
ops = {'+': lambda x,y: x+y,
       '-': lambda x,y: x-y,
       '*': lambda x,y: x*y,
       '/': division,
       '^': potencia}

In [ ]:
#Valores aceptados en la Notación Polaca
valores_aceptados = '0123456789+-*/^. '

def transformar(exp):
  """
    Extrae el primer dígito de la cadena en Notación Polaca.
    Si es un número, lo vuelve un float.
    Si es un operador, lo deja igual.

    Args:
      Expresión de Notación Polaca (str)

    Returns:
      Número u operador que está al inicio de la cadena.
      Cadena original recortada a partir del siguiente valor.
  """
  if ' ' not in exp:
    valor, exp = exp, ''
  else:
    i = 0
    while exp[i] != ' ':
      #Revisa que no hayan caracteres diferentes a los aceptados en la notación polaca
      if exp[i] not in valores_aceptados: raise Exception(f'El caracter {exp[i]} no es válido')
      else: i += 1

    valor = exp[:i] #Guarda el valor inicial de la cadena
    exp = exp[i+1:] #Recorta la cadena original para que empiece desde el siguiente caracter.

  if valor in ops: return valor, exp
  else: return float(valor), exp

In [ ]:
def Notacion_Polaca(exp):
    """Notación Polaca
    >>> Notacion_Polaca("2 3 +")
    5
    >>> Notacion_Polaca("2 3 ^")
    8
    >>> Notacion_Polaca("4 0.5 ^")
    2
    >>> Notacion_Polaca("2 -1 ^")
    0.5
    >>> Notacion_Polaca("5 2 + 8 3 - * 4 ^")
    1500625
    """

    """
    Resuelve una operación en Notación Polaca.
    En un Dynamic Array se guardan uno a uno los valores de la expresión hasta llegar a un operador.
    Cuando pasa esto, se extraen del array los últimos 2 valores y se operan.
    El resultado se guarda nuevamente en el Array.
    Esto sucesivamente hasta que en la expresión ya no hayan más valores.

    Args:
        Expresión de Notación Polaca (str)

    Returns:
        Número entero o float resultado de la operación Notación Polaca.
    """
    array_np = DynamicArray()

    while exp:
      valor_actual, exp = transformar(exp) #Se guardan sucesivamente los valores de la exp de acuerdo a la función de arriba

      if valor_actual in ops:
        num_2, num_1 = array_np.pop(), array_np.pop() #Extrae los últimos dos valores del array
        valor_actual = ops[valor_actual](num_1, num_2) #Se actualiza 'valor actual'. Los opera llamando al diccionario 'ops' según el operador.

      array_np.append(valor_actual)

    if array_np.__len__() != 1: #Revisa que no hayan más valores por operar. De no ser así, la notación polaca ingresada no es válida
      raise Exception('La operación en Notación Polaca no se puede realizar')

    final = array_np.__getitem__(0)
    if int(final) - final == 0: return int(final)
    else: return final

import doctest
doctest.testmod()

TestResults(failed=0, attempted=5)